<a href="https://colab.research.google.com/github/LongNguyen1984/TimeSeriesWithPython/blob/main/EnsembleModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# argmax function
def argmax(vector):
  index, value = 0, vector[0]
  for i, v in enumerate(vector):
    if v > value:
      index, value = i,v
  return index

# define vector 
vector = [0.4, 0.5 ,0.1]
print('arg max of %s: %d'%(vector, argmax(vector)))

arg max of [0.4, 0.5, 0.1]: 1


In [ ]:
# model averaging ensemble and a study of ensemble size on test accuracy
from sklearn.datasets import make_blobs
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy
from numpy import array
from numpy import argmax
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
 
# fit model on dataset
def fit_model(trainX, trainy):
	# define model
	model = Sequential()
	model.add(Dense(15, input_dim=2, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit model
	model.fit(trainX, trainy, epochs=200, verbose=0)
	return model
 
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, testX):
	# make predictions
  yhats = [model.predict(testX) for model in members]
  yhats = array(yhats)
  #print(yhats)
  # sum across ensemble members
  summed = numpy.sum(yhats, axis=0)
	# argmax across classes
  result = argmax(summed, axis=1)
  return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(members, n_members, testX, testy):
	# select a subset of members
	subset = members[:n_members]
	print(len(subset))
	# make prediction
	yhat = ensemble_predictions(subset, testX)
	# calculate accuracy
	return accuracy_score(testy, yhat)
 
# generate 2d classification dataset
X, y = make_blobs(n_samples=500, centers=3, n_features=2, cluster_std=2, random_state=2)
# split into train and test
n_train = int(0.3 * X.shape[0])
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
trainy = to_categorical(trainy)
# fit all models
n_members = 20
members = [fit_model(trainX, trainy) for _ in range(n_members)]
# evaluate different numbers of ensembles
scores = list()
for i in range(1, n_members+1):
	score = evaluate_n_members(members, i, testX, testy)
	print('> %.3f' % score)
	scores.append(score)
# plot score vs number of ensemble members
x_axis = [i for i in range(1, n_members+1)]
pyplot.plot(x_axis, scores)
pyplot.show()



In [28]:
yhats = [ members[i].predict(testX) for i in range(0,2)]
yhats = array(yhats)

In [34]:
import numpy as np


summed = np.sum(yhats, axis=0)

In [32]:
np.shape(yhats)

(2, 350, 3)

In [35]:
np.shape(summed), summed

((350, 3), array([[7.3212802e-01, 3.8570255e-01, 8.8216925e-01],
        [1.1392098e-03, 1.9987142e+00, 1.4637104e-04],
        [4.3660045e-01, 3.7906772e-01, 1.1843319e+00],
        ...,
        [3.3777374e-01, 1.0990586e+00, 5.6316769e-01],
        [6.6449061e-02, 1.1954141e+00, 7.3813683e-01],
        [1.9643010e-01, 1.4158821e+00, 3.8768786e-01]], dtype=float32))

In [16]:
testX[0:10]

array([[ -0.30956392,  -4.87316732],
       [  4.81845091,  -1.38407855],
       [ -0.68524041,  -4.15563282],
       [ -0.27433912, -11.97205154],
       [  1.75965848,  -2.12754045],
       [ -0.7972227 ,  -8.48662407],
       [ -0.28758826,  -1.47819873],
       [  5.64603136,  -2.5616584 ],
       [ -0.77216809,   0.47593624],
       [  3.0215866 ,  -4.66238588]])